In [1]:
import os
import numpy as np
import tqdm
import json
import random
import time
import multiprocessing
from ember_features import PEFeatureExtractor

from datetime import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import operator
import copy


In [6]:
def get_emberdata_family_stat(indicate_month):
    data_dir = "../../../ember2017/ember_2017_2/"
    raw_feature_paths_base_tr = [os.path.join(data_dir, "train_features_{}.jsonl".format(i)) for i in range(6)]
    raw_feature_paths_base_te = [os.path.join(data_dir, "test_features.jsonl")]
    raw_feature_paths = raw_feature_paths_base_tr + raw_feature_paths_base_te
    #print(raw_feature_paths)

    all_task_months = ['2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
                   '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12']
    task_months = all_task_months[indicate_month]

    av_class_stats = {}
    cnt_rows = 0
    cnt_good_rows = 0
    cnt_missing_rows = 0

    for fp in raw_feature_paths:
        #print(fp)
        with open(fp, "r") as fin:
            #print(fp)
            for line in fin:
                raw_features = json.loads(line)
                #print(raw_features.keys())

                if raw_features['appeared'] in task_months:
                    if raw_features['label'] == 1: # and raw_features['avclass']
                        #print(raw_features['label'], raw_features['avclass'])
                        try:
                            if raw_features['avclass'] not in av_class_stats.keys():
                                av_class_stats[raw_features['avclass']] = 1
                            else:
                                av_class_stats[raw_features['avclass']] += 1
                        except:
                            if 'missing_avclass' not in av_class_stats.keys():
                                av_class_stats['missing_avclass'] = 1
                            else:
                                av_class_stats['missing_avclass'] += 1
                                
                                
                        cnt_rows += 1

                    elif raw_features['label'] == 0:
                        cnt_good_rows += 1
                    elif raw_features['label'] == -1:
                        #print(raw_features['label'], raw_features['avclass'])
                        cnt_missing_rows += 1

            #if cnt_rows == 2:
            #    break
    
    
    
    min_samples = 0
    
    families_more_than_400_samples = {}

    for k, v in av_class_stats.items():
        if v >= min_samples and k != '':
            families_more_than_400_samples[k] = v
    
    
    stat_list = np.array(list(av_class_stats.values()))
    #print(f' Samples {sum(stat_list)} families {len(av_class_stats.keys())}')
    print(f'Month {task_months} has {cnt_rows} malware samples \n belonging to {len(av_class_stats.keys())} families, goodware {cnt_good_rows}') 
    return families_more_than_400_samples, av_class_stats

#families_more_than_400_samples, av_class_stats = get_emberdata_family_stat(0)

In [ ]:
for i in range(12):
    families_more_than_400_samples, av_class_stats = get_emberdata_family_stat(i)
    print()
    print('********************')
    print()

Month 2017-01 has 32761 malware samples 
 belonging to 1 families, goodware 17180

********************



In [ ]:
stat_list = np.array(list(av_class_stats.values()))
print(sum(stat_list))

min_samples = 0

valid_families = {}

for k, v in av_class_stats.items():
    if v >= min_samples and k != '':
        valid_families[k] = v
        
print(sum(np.array(list(valid_families.values()))), len(np.array(list(valid_families.keys()))))

In [ ]:
min_samples = 400

valid_families = {}

for k, v in av_class_stats.items():
    if v >= min_samples and k != '':
        valid_families[k] = v
print(sum(np.array(list(valid_families.values()))), len(np.array(list(valid_families.keys()))))

In [ ]:
2899 - 106

In [ ]:
stat_list = np.array(list(av_class_stats.values()))

stat_list = [i for i in stat_list if i > 500]
plt.figure(figsize=(20, 6))
plt.bar(range(len(stat_list)),stat_list)

In [ ]:
#stat_list

In [ ]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 15
#plt.rcParams['font.family'] = "serif"
tdir = 'in'
major = 5.0
minor = 3.0
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor



min_samples = 400


families_more_than_400_samples = {}

labels = []
num_samples = []



for k, v in stat_ember:
    if v >= min_samples and k != '':
        #print(k, v)
        labels.append(k)
        num_samples.append(v)
        families_more_than_400_samples[k] = v
        
print(len(labels), len(num_samples))

'''
save_file_name = 'ember_' + str(min_samples) + '_stats.png'
#print(x_axis, y_)
plt.figure(figsize=(20, 6))
plt.bar(range(len(labels)), num_samples, color='blue')
plt.xticks(range(len(labels)), labels, rotation=90)
plt.ylabel('Number of Samples', fontsize=20)
plt.xlabel(f'Family Labels', fontsize=20)
#plt.grid(True)
plt.savefig('./store_figures/' + save_file_name,
        bbox_inches='tight', 
       transparent=True);
       
'''

In [ ]:
# family, bad, good, unknown
#(2900, 400000, 350000, 200000)
# 11433 missing avclass

In [ ]:
def vectorize(irow, raw_features_string, X_path, y_path, extractor, nrows):
    """
    Vectorize a single sample of raw features and write to a large numpy file
    """
    raw_features = json.loads(raw_features_string)
    
    feature_vector = extractor.process_raw_features(raw_features)

    y = np.memmap(y_path, dtype=np.float32, mode="r+", shape=nrows)
    y[irow] = top_families_100_labels[raw_features["avclass"]]

    X = np.memmap(X_path, dtype=np.float32, mode="r+", shape=(nrows, extractor.dim))
    X[irow] = feature_vector


def vectorize_unpack(args):
    """
    Pass through function for unpacking vectorize arguments
    """
    return vectorize(*args)



def create_parent_folder(file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
        
def raw_feature_iterator(file_paths, top_families):
    """
    Yield raw feature strings from the inputed file paths
    """
    all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']
    
    for path in file_paths:
        with open(path, "r") as fin:
            for line in fin:
                raw_features = json.loads(line)
                if raw_features['appeared'] in all_task_months:
                    if raw_features['avclass'] != '':
                        if raw_features['avclass'] in top_families and raw_features['label'] == 1:
                            yield line


def task_based_vectorize_subset(X_path, y_path, raw_feature_paths, top_families, extractor, nrows):
    """
    Vectorize a subset of data and write it to disk
    """
    # Create space on disk to write features to
    X = np.memmap(X_path, dtype=np.float32, mode="w+", shape=(nrows, extractor.dim))
    y = np.memmap(y_path, dtype=np.float32, mode="w+", shape=nrows)
    del X, y

    # Distribute the vectorization work
    pool = multiprocessing.Pool()
    argument_iterator = ((irow, raw_features_string, X_path, y_path, extractor, nrows)
                         for irow, raw_features_string in enumerate(raw_feature_iterator(raw_feature_paths, top_families)))
    #print(argument_iterator)
    
    
    for _ in tqdm.tqdm(pool.imap_unordered(vectorize_unpack, argument_iterator), total=nrows):
        pass
    
    #return argument_iterator

        
def task_num_rows(raw_feature_paths, top_families):
    print(top_families)
    all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']
    cnt_rows = 0
    for fp in raw_feature_paths:
        #print(fp)
        with open(fp, "r") as fin:
            #print(fp)
            for line in fin:
                raw_features = json.loads(line)
                if raw_features['appeared'] in all_task_months:
                    if raw_features['avclass'] != '':
                        if raw_features['avclass'] in top_families and raw_features['label'] == 1:
                            cnt_rows += 1
    return cnt_rows


def create_task_based_vectorized_features(data_dir, save_dir, top_families, feature_version=2):
    """
    Create feature vectors from raw features and write them to disk
    """
    extractor = PEFeatureExtractor(feature_version)
    
    #print(f'Vectorizing {current_task} task data')
    X_path = os.path.join(save_dir, "X_train.dat")
    y_path = os.path.join(save_dir, "y_train.dat")
    raw_feature_paths_base_tr = [os.path.join(data_dir, "train_features_{}.jsonl".format(i)) for i in range(6)]
    raw_feature_paths_base_te = [os.path.join(data_dir, "test_features.jsonl")]
    raw_feature_paths = raw_feature_paths_base_tr + raw_feature_paths_base_te
    
    
    
    nrows = task_num_rows(raw_feature_paths, top_families)
    #print(nrows)
    task_based_vectorize_subset(X_path, y_path, raw_feature_paths, top_families, extractor, nrows)
    #argument_iterator = task_based_vectorize_subset(X_path, y_path, raw_feature_paths, task_months, extractor, nrows)
    
    #return argument_iterator

def read_task_based_vectorized_features(save_dir, feature_version=2):
    """
    Read vectorized features into memory mapped numpy arrays
    """

    extractor = PEFeatureExtractor(feature_version)
    ndim = extractor.dim
    X_ = None
    y_ = None


    X_path = os.path.join(save_dir, "X_train.dat")
    y_path = os.path.join(save_dir, "y_train.dat")
    
    y_ = np.memmap(y_path, dtype=np.float32, mode="r")
    N = y_.shape[0]
    
    X_ = np.memmap(X_path, dtype=np.float32, mode="r", shape=(N, ndim))
    
    print(np.unique(y_))
    
    X, Y = X_, y_
    
    indx = [i for i in range(len(Y))]
    random.shuffle(indx)

    train_size = int(len(indx)*0.9)
    trainset = indx[:train_size]
    testset = indx[train_size:]

    # Separate the training set
    X_train = X[trainset]
    Y_train = Y[trainset]

    # Separate the test set
    X_test = X[testset]
    Y_test = Y[testset]
    
    
    print(f'X_train {X_train.shape} Y_train {Y_train.shape} X_test {X_test.shape} Y_test {Y_test.shape}')
    
    print(f'saving files ...')
    save_training_file = save_dir + 'XY_train.npz'
    save_test_file = save_dir + 'XY_test.npz'
    
    np.savez(save_training_file, X_train=X_train, Y_train=Y_train)
    np.savez(save_test_file, X_test=X_test, Y_test=Y_test)
    

In [ ]:
#ordered_106_families

In [ ]:
#ordered_100_families_keys_100
families_more_than_400_samples = get_emberdata_family_stat()
ordered_106_families = sorted(families_more_than_400_samples.items(),key=operator.itemgetter(1),reverse=True)


ordered_100_families_keys_100 = []
num_classes = 100

cnt = 0
for j in ordered_106_families:
    #print(j[0], j[1])
    cnt += 1
    ordered_100_families_keys_100.append(j[0])
    if cnt == num_classes:
        break
print(len(ordered_100_families_keys_100))

    
top_families_100_labels = {}

for ind, fam in enumerate(ordered_100_families_keys_100):
    top_families_100_labels[fam] = int(ind)
    
    
all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

data_dir = "../../ember/ember_data/2018_data/ember2018/"


start_time = time.time()

#top_families = ordered_100_families_keys_100

save_dir = '../../ember2018/top_class_bases/top_classes_' + str(num_classes) + '/'

create_parent_folder(save_dir)


create_task_based_vectorized_features(data_dir, save_dir, ordered_100_families_keys_100, feature_version=2)
read_task_based_vectorized_features(save_dir, feature_version=2)
    
    
end_time = time.time()

print(f'Elapsed time {(end_time - start_time)/60} mins.')

In [ ]:
# 50 class X_train (269810, 2381) Y_train (269810,) X_test (29979, 2381) Y_test (29979,)
# 100 class 

In [ ]:
extractor = PEFeatureExtractor(2)

#print(f'Vectorizing {current_task} task data')
X_path = os.path.join(save_dir, "X_train.dat")
y_path = os.path.join(save_dir, "y_train.dat")
raw_feature_paths_base_tr = [os.path.join(data_dir, "train_features_{}.jsonl".format(i)) for i in range(6)]
raw_feature_paths_base_te = [os.path.join(data_dir, "test_features.jsonl")]
raw_feature_paths = raw_feature_paths_base_tr + raw_feature_paths_base_te

In [ ]:
top_families = ordered_100_families_keys_100[:3]
top_families

In [ ]:
all_task_months

In [ ]:
def task_num_rows(raw_feature_paths, top_families):
    print(top_families)
    cnt_rows = 0
    for fp in raw_feature_paths:
        #print(fp)
        with open(fp, "r") as fin:
            #print(fp)
            for line in fin:
                raw_features = json.loads(line)
                if raw_features['appeared'] in all_task_months:
                    if raw_features['avclass'] != '':
                        if raw_features['avclass'] in top_families and raw_features['label'] == 1:
                            cnt_rows += 1
    return cnt_rows


#nrows = task_num_rows(raw_feature_paths, ordered_100_families_keys_100[0])
#print(nrows)

for i in range(5):
    #print(ordered_100_families_keys_100[i])
    nrows = task_num_rows(raw_feature_paths, ordered_100_families_keys_100[i])
    print(nrows)
#task_based_vectorize_subset(X_path, y_path, raw_feature_paths, top_families, extractor, nrows)

In [ ]:
xtrat samples 35969
zbot samples 24075
ramnit samples 20595
sality samples 18572
installmonster samples 16691

In [ ]:
ordered_100_families_keys_100[0]

In [ ]:
data_dir = "../../ember/ember_data/2018_data/ember2018/"
raw_feature_paths_base_tr = [os.path.join(data_dir, "train_features_{}.jsonl".format(i)) for i in range(6)]
raw_feature_paths_base_te = [os.path.join(data_dir, "test_features.jsonl")]
raw_feature_paths = raw_feature_paths_base_tr + raw_feature_paths_base_te
#print(raw_feature_paths)

all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

task_months = all_task_months

cnt_rows = 0


for fp in raw_feature_paths:
    #print(fp)
    with open(fp, "r") as fin:
        #print(fp)
        for line in fin:
            raw_features = json.loads(line)
            #print(raw_features.keys())

            if raw_features['appeared'] in task_months:
                if raw_features['avclass'] != '':
                    if raw_features['label'] == 1 and raw_features['avclass'] in ordered_100_families_keys_100[4:5]:
                        cnt_rows += 1
print(cnt_rows)